In [ ]:
!pip -q install -U langchain langchain-google-genai pydantic python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 992.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 25.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving .env to .env


In [ ]:
!ls -la

total 20
drwxr-xr-x 1 root root 4096 Aug 28 10:21 .
drwxr-xr-x 1 root root 4096 Aug 28 10:18 ..
drwxr-xr-x 4 root root 4096 Aug 26 19:08 .config
-rw-r--r-- 1 root root   54 Aug 28 10:21 .env
drwxr-xr-x 1 root root 4096 Aug 26 19:08 sample_data


In [ ]:
from dotenv import load_dotenv, find_dotenv
import os

load_dotenv(find_dotenv())

key = os.getenv("GOOGLE_API_KEY")
print("API Key loaded?", bool(key))
print("Key prefix:", (key[:6] + "…") if key else None)

API Key loaded? True
Key prefix: AIzaSy…


In [ ]:
import sys
from typing import List
from pydantic import BaseModel, ValidationError, field_validator
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

from datetime import datetime

class ResearchBrief(BaseModel):
    title: str
    problem_statement: str
    key_questions: List[str]
    method_brief: List[str]
    deliverables: List[str]
    generated_at: str  # field for date & time

    @field_validator("problem_statement")
    @classmethod
    def max_two_sentences(cls, v: str) -> str:
        count = sum(v.count(ch) for ch in ".!?")
        if count > 2:
            raise ValueError("problem_statement must be <= 2 sentences")
        return v.strip()

    @field_validator("key_questions")
    @classmethod
    def q_len(cls, v: List[str]) -> List[str]:
        if not (1 <= len(v) <= 3):
            raise ValueError("key_questions must have 1–3 items")
        return [s.strip() for s in v]

    @field_validator("method_brief")
    @classmethod
    def m_len(cls, v: List[str]) -> List[str]:
        if not (2 <= len(v) <= 4):
            raise ValueError("method_brief must have 2–4 items")
        return [s.strip() for s in v]

    @field_validator("deliverables")
    @classmethod
    def d_len(cls, v: List[str]) -> List[str]:
        if not (2 <= len(v) <= 3):
            raise ValueError("deliverables must have 2–3 items")
        return [s.strip() for s in v]


In [ ]:
def add_timestamp(brief: ResearchBrief) -> ResearchBrief:
    brief.generated_at = datetime.now().strftime("%Y-%m-%d")
    return brief

In [ ]:
def require_api_key():
    if not os.getenv("GOOGLE_API_KEY"):
        raise RuntimeError(
            "Missing GOOGLE_API_KEY. Set it using:\n"
            "os.environ['GOOGLE_API_KEY'] = 'your_key_here'"
        )

def build_chain():
    parser = PydanticOutputParser(pydantic_object=ResearchBrief)

    system_rules = (
        "You write ultra-concise, practical research briefs.\n"
        "Return ONLY JSON that matches this schema:\n{format_instructions}\n"
        "Rules:\n"
        "- Keep 'problem_statement' to <= 2 sentences.\n"
        "- 'key_questions' has 1–3 items.\n"
        "- 'method_brief' has 2–4 items.\n"
        "- 'deliverables' has 2–3 items.\n"
        "- Do NOT include date or time (will be added later).\n"
        "No markdown, no extra keys, no commentary."
    )

    prompt = (
        ChatPromptTemplate
        .from_messages([
            ("system", system_rules),
            ("human", "Topic: {topic}")
        ])
        .partial(format_instructions=parser.get_format_instructions())
    )

    model = ChatGoogleGenerativeAI(
        model="gemini-1.5-flash",
        temperature=0,
        max_retries=2,
    )


    return prompt | model | parser | add_timestamp


def to_markdown(brief: ResearchBrief) -> str:
    lines = []
    lines.append(f"# {brief.title}")
    lines.append(f"**Generated At:** {brief.generated_at}")
    lines.append(f"**Problem:** {brief.problem_statement}")
    lines.append("**Key Questions:**")
    lines.extend(f"- {q}" for q in brief.key_questions)
    lines.append("**Method (brief):**")
    lines.extend(f"- {m}" for m in brief.method_brief)
    lines.append("**Deliverables:**")
    lines.extend(f"- {d}" for d in brief.deliverables)
    return "\n".join(lines)



In [ ]:
require_api_key()
topic = input("Enter a research topic: ").strip()

if not topic:
    print("ERROR: Topic cannot be empty.", file=sys.stderr)
else:
    try:
        chain = build_chain()
        brief: ResearchBrief = chain.invoke({"topic": topic})

        print("\n JSON Output:")
        print(brief.model_dump_json(indent=2, ensure_ascii=False))

    except ValidationError as ve:
        print("ERROR: Model returned invalid JSON.", file=sys.stderr)
        print(ve, file=sys.stderr)
    except Exception as e:
        print(f"ERROR: {e}", file=sys.stderr)


Enter a research topic: Ai impact on human

 JSON Output:


ERROR: BaseModel.model_dump_json() got an unexpected keyword argument 'ensure_ascii'


In [ ]:
import json
from datetime import datetime

current_time = datetime.now().strftime("%Y-%m-%d")

result = chain.invoke({"topic": topic})

output_data = result.model_dump()
output_data["generated at"] = current_time

json_output = json.dumps(output_data, indent=2, ensure_ascii=False)
print("JSON Output:")
print(json_output)

md_output = to_markdown(result)
md_output += f"\n\n**Generated on:** {current_time}"

print("\nMarkdown Output:")
print(md_output)


JSON Output:
{
  "title": "AI Impact on Human Labor",
  "problem_statement": "AI-driven automation is rapidly changing the job market.  This research will assess the extent and nature of this impact on human workers.",
  "key_questions": [
    "How will AI affect employment rates across various sectors?",
    "What types of jobs are most susceptible to automation?",
    "What retraining and upskilling initiatives are needed?"
  ],
  "method_brief": [
    "Literature review of existing studies on AI and employment.",
    "Statistical analysis of job market trends and AI adoption rates.",
    "Case studies of companies implementing AI and their workforce adjustments.",
    "Interviews with workers in AI-affected industries."
  ],
  "deliverables": [
    "A report summarizing the findings of the research.",
    "Policy recommendations for mitigating negative impacts of AI on employment.",
    "A presentation of key findings for stakeholders."
  ],
  "generated_at": "2025-08-28",
  "genera